In [5]:
import json 

with open('03-evaluation/documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [8]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200", verify_certs=False)
print(es_client.info())

{'name': '8ec90d082f04', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'rwgqZ9DaRmiFZ_R17ltknQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "analysis": {
            "analyzer": {
                "default": { "type": "standard" }
            }
        }
    },
    "mappings": {
        "properties": {
            "id": {"type": "keyword"},
            "text": {"type": "text"},
            "section": {"type": "keyword"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}
index_name = "course-questions"

if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [10]:
print(es_client.info())

{'name': '8ec90d082f04', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'rwgqZ9DaRmiFZ_R17ltknQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [11]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:04<00:00, 213.95it/s]


In [12]:
def elastic_search(query, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must" : {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [13]:
search_response = elastic_search(
    query="I just discovered the course. Can I still join?",
    course="data-engineering-zoomcamp"
)

In [14]:
import pandas as pd 

df_ground_truth = pd.read_csv('03-evaluation/ground-truth-data.csv')
df_ground_truth.head()

,question,course,document
0,When does the course begin?,data-engineering-zoomcamp,c02e79ef
1,How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
2,What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
3,How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
4,Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef


In [15]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [16]:
from tqdm.auto import tqdm
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = elastic_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [00:15<00:00, 291.15it/s]


In [17]:
def hit_rate(relevance):
    cnt = 0
    for line in relevance:
        if True in line:
            cnt += 1
    return cnt / len(relevance)

In [97]:
# def mrr(relevance_total):
#     total_score = 0.0

#     for line in relevance_total:
#         for rank in range(len(line)):
#             if line[rank] == True:
#                 total_score = total_score + 1 / (rank + 1)

#     return total_score / len(relevance_total)

In [18]:
def mrr(relevance: list[list[bool]]) -> float:
    total_score = 0.0
    for line in relevance:
        try:
            rank = line.index(True)
            total_score += 1.0 / (rank + 1)
        except ValueError:
            pass
    return total_score / len(relevance) if relevance else 0.0

In [19]:
hit_rate(relevance_total), mrr(relevance_total)

(0.7395720769397017, 0.6024385851163465)

In [20]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course", "id"]
)

index.fit(documents)

/workspaces/llm-zoomcamp/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [21]:
def minsearch_search(query, course):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': course},
        boost_dict=boost,
        num_results=5
    )

    return results

In [22]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = minsearch_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [00:14<00:00, 319.23it/s]


In [23]:
hit_rate(relevance_total), mrr(relevance_total)

(0.7722066133563864, 0.6609862401844251)

Compare with ES result: 
```
(0.7395720769397017, 0.6024385851163465)
```

In [24]:
def evaluate(ground_truth, search_function):
    relevant_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)
    metrics = {
        "hit_rate": hit_rate(relevance_total), 
        "mrr": mrr(relevance_total)
    }
    return metrics

In [25]:
evaluate(ground_truth, lambda q : elastic_search(q['question'], q['course']))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [00:11<00:00, 419.18it/s]


{'hit_rate': 0.7558893451480441, 'mrr': 0.6317124126503793}